In [98]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections
from importlib import reload
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [143]:
from surprise import Reader, Dataset, SVD, NormalPredictor
# from surprise import evaluate
import surprise.model_selection

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import scipy.stats
from sklearn import preprocessing
import sklearn.cluster
import keras
import keras.utils
from keras import optimizers, backend as K
from keras.layers import *
from keras.models import Model

In [144]:
import util.plot
import util.data
from util import clustering

In [145]:
reload(util.plot)
reload(util.data);

In [146]:
data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=50*1000)
util.data.rm_na(data_all)
data_all.shape

(50000, 85)

In [147]:
data, data_test = util.data.train_test_split(data_all)

# split cross validation folds
folds = util.data.cv_folds_for_sklearn(data, n_cv_folds=3, resampling_ratio=0)

# Clustering

See also https://blog.keras.io/building-autoencoders-in-keras.html and https://github.com/voschezang/drum-style-transfer

In [148]:
# a = data_all.iloc[:10] # ['cluster_id_items_KMeans']
# # a.loc[:, 'cluster_id_items_KMeans'] = 3
# indices = list(a.index)
# indices += indices
# b = data_all.loc[indices]
# # len(list(a.index))
# # data_all.loc[indices].shape, len(list(indices))
# # b.loc[indices].shape, len(indices), len(list(indices))
# a.loc[[0,1], 'prop_brand_bool'] = [4,5]
# data_all

In [177]:
# suppress warning to improve speed
pd.options.mode.chained_assignment = None  # default='warn'
reload(util.data)
reload(util.clustering)
# cluster_id_items_KMeans

cv_results = collections.defaultdict(list)
for i_train, i_test in folds:
    # Cluser users & items
    keys_search, keys_property, models_user, models_item = clustering.init(data_all)
    clustering.init_df_columns(data_all, models_user, models_item)
    xy_train = data_all.loc[i_train]
#     clustering.train(xy_train, keys_search, keys_property, models_user, models_item)
    clustering.fit(xy_train, models_user, keys_search,'srch_id')
    clustering.fit(xy_train, models_item, keys_property,'prop_id')

    users = clustering.predict(data_all, models_user, keys_search,
                 'srch_id', clustering.USER_KEY_PREFIX)
    items = clustering.predict(data_all, models_item, keys_property,
                 'prop_id', clustering.ITEM_KEY_PREFIX)

    for k in users.columns:
        util.data.replace_most_uncommon(users, k)
        data_all.loc[users.index, k] = users[k]
    for k in items.columns:
        util.data.replace_most_uncommon(items, k)
        data_all.loc[items.index, k] = items[k]
    
    assert not items.isna().any().any()
    
    # train SVD's

    # check all combinations
    for k_user in users.columns:
        for k_item in items.columns:
            print(k_user, k_item)
            assert not data_all[k_user].isna().any()
            assert not data_all[k_item].isna().any()            
            
            scores = util.data.scores_df(data_all, k_user, k_item)
            # check minimal occurence
            value_counts_user = scores.user.value_counts(ascending=True)
            value_counts_item = scores.item.value_counts(ascending=True)
            assert value_counts_user.iloc[0] > 2, value_counts_user # use more data?
            assert value_counts_item.iloc[0] > 2, value_counts_item
            
            reader = Reader(rating_scale=(0,6))
            data_scores = Dataset.load_from_df(scores, reader)
            # surprise.model_selection.cross_validate(SVD(), data_scores, cv=5)
            # model = SVD(**gs.best_params['rmse'])
            model = SVD()
            trainset, testset = surprise.model_selection.train_test_split(data_scores, test_size=0.3, random_state=seed)
            model.fit(trainset)
            predictions = model.test(testset)
            rmse,fcp = surprise.accuracy.rmse(predictions), surprise.accuracy.fcp(predictions)
            cv_results[k_user + '-' + k_item].append(surprise.accuracy.rmse(predictions))
#             cv_result[k_user + '-' + k_item]['fcp'].append(surprise.accuracy.fcp(predictions))
            print('\t rmse: %0.2f , fcp: %0.2f' % (rmse, fcp))        

	extract_data(k: srch_id)
	extract_data(k: prop_id)
	KMeans (k: `cluster_id_users_KMeans`)
	FeatureAgglomeration (k: `cluster_id_users_FeatureAgglomeration`)
	KMeans (k: `cluster_id_items_KMeans`)
	FeatureAgglomeration (k: `cluster_id_items_FeatureAgglomeration`)
cluster_id_users_KMeans cluster_id_items_KMeans
RMSE: 0.8289
FCP:  0.4615
RMSE: 0.8289
	 rmse: 0.83 , fcp: 0.46
cluster_id_users_KMeans cluster_id_items_FeatureAgglomeration
RMSE: 0.8282
FCP:  0.4431
RMSE: 0.8282
	 rmse: 0.83 , fcp: 0.44
cluster_id_users_FeatureAgglomeration cluster_id_items_KMeans
RMSE: 0.6015
FCP:  0.9644
RMSE: 0.6015
	 rmse: 0.60 , fcp: 0.96
cluster_id_users_FeatureAgglomeration cluster_id_items_FeatureAgglomeration
RMSE: 0.6021
FCP:  0.9645
RMSE: 0.6021
	 rmse: 0.60 , fcp: 0.96
	extract_data(k: srch_id)
	extract_data(k: prop_id)
	KMeans (k: `cluster_id_users_KMeans`)
	FeatureAgglomeration (k: `cluster_id_users_FeatureAgglomeration`)
	KMeans (k: `cluster_id_items_KMeans`)
	FeatureAgglomeration (k: `cluster_

In [179]:
# reload(util.data)
# i_train, i_test = folds[0]
# xy_train = data_all.loc[i_train]
# if True:
#     # check all combinations
#     for k_user in users.columns:
#         for k_item in items.columns:
#             print(k_user, k_item)
#             scores = util.data.scores_df(xy_train, k_user, k_item)
#             # check minimal occurence
#             value_counts_user = scores.user.value_counts(ascending=True)
#             value_counts_item = scores.item.value_counts(ascending=True)
#             assert value_counts_user.iloc[0] > 2 # use more data?
#             assert value_counts_item.iloc[0] > 2
            
#             reader = Reader(rating_scale=(0,6))
#             data_scores = Dataset.load_from_df(scores, reader)
#             # surprise.model_selection.cross_validate(SVD(), data_scores, cv=5)
#             # model = SVD(**gs.best_params['rmse'])
#             model = SVD()
#             trainset, testset = surprise.model_selection.train_test_split(data_scores, test_size=0.3, random_state=seed)
#             model.fit(trainset)
#             predictions = model.test(testset)
#             rmse,fcp = surprise.accuracy.rmse(predictions), surprise.accuracy.fcp(predictions)
#             cv_result[k_user + '-' + k_item].append(surprise.accuracy.rmse(predictions))
# #             cv_result[k_user + '-' + k_item]['fcp'].append(surprise.accuracy.fcp(predictions))
#             print('\t rmse: %0.2f , fcp: %0.2f' % (rmse, fcp))

In [182]:
for k, values in cv_result.items():
    print('%s & %0.4f & %0.4f' % (k,np.mean(values), np.std(values)))

cluster_id_users_KMeans-cluster_id_items_KMeans & 0.828 & 0.0004
cluster_id_users_KMeans-cluster_id_items_FeatureAgglomeration & 0.828 & 0.0003
cluster_id_users_FeatureAgglomeration-cluster_id_items_KMeans & 0.602 & 0.0006
cluster_id_users_FeatureAgglomeration-cluster_id_items_FeatureAgglomeration & 0.602 & 0.0000


In [111]:
def predict(model, dataset):
    results = collections.defaultdict(dict)
    for item, user, score in testset:
        result = model.predict(str(item), str(user), verbose=0)
        results[user][item] = result.est
    return results
# surprise.SVD.predict()

In [112]:
scores_pred = predict(model, testset)
# scores_pred

In [113]:
# TODO use original user/item ids and convert results to csv
# i.e. 
result = collections.defaultdict(dict)
for row in 'train.csv':
    user_id = kmeans_model.predict(row[keys_search])
    item_id = kmeans_model.predict(row[keys_property])
    score = predict(model, (user_id, item_id))
    result[user_id][item_id] = score
    
# then convert result dict to df with columns (user id, prop id)

NameError: name 'kmeans_model' is not defined

In [ ]:
# surprise.model_selection.cross_validate(SVD(), data_scores, cv=5)

In [32]:
# y_train = y_train.toarray()
# y_train = y_train[:,:,np.newaxis]
# y_train.shape

In [17]:
# x_train = np.random.random(x_train.shape)
# y_train = np.random.random(y_train.shape)

In [16]:
# n_dims = data.shape[1]
input_shape = x_train.shape[1:]
output_shape = x_train.shape[1:]
print(input_shape, output_shape)

(35, 1) (35, 1)


In [33]:
encoder_input = Input(shape=input_shape)
h = encoder_input
h = Flatten()(h)
h = Dense(100, activation='relu')(h)
# decoder
h = Dense(np.prod(output_shape), activation='relu')(h)
h = Reshape(output_shape)(h)
encoder_output = Activation('sigmoid')(h)
model = Model(encoder_input, encoder_output, name='ae-')
model.compile(optimizer='adam', loss='binary_crossentropy') # RMSprop Adadelta SGD

(35, 1) (35, 1)


In [17]:
input_shape, output_shape

((35, 1), (35, 1))

In [36]:
# validation_split does not shuffle the data
result = model.fit(x_train, x_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# history = result.history

Train on 32 samples, validate on 8 samples
Epoch 1/2
32/32 [==============================] - 1s 33ms/step - loss: -35.1065 - val_loss: -38.4029
Epoch 2/2
32/32 [==============================] - 0s 433us/step - loss: -35.2843 - val_loss: -38.4029


In [37]:
# result.params
result.history

{'val_loss': [-38.40291213989258, -38.40291213989258],
 'loss': [-35.10652494430542, -35.28428649902344]}

In [22]:
# model.predict(x_train)

In [38]:
# vae, encoder, decoder = init_vae(input_shape, output_shape, n_latent_dims=10)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 35, 1)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 35)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               3600      
_________________________________________________________________
dense_10 (Dense)             (None, 35)                3535      
_________________________________________________________________
reshape_4 (Reshape)          (None, 35, 1)             0         
_________________________________________________________________
activation_2 (Activation)    (None, 35, 1)             0         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________


In [18]:
# keras.utils.plot_model(model, to_file='model.pdf', show_shapes=True, rankdir='TB') # TB LR

In [19]:
# First design an AE as simple as possible, to avoid overfitting
# I.e. reduce the number of nodes until the (train) performance drops
# Then increase the complexity (i.e. number of layers) to improve the compression

In [20]:
# see also
# https://github.com/voschezang/drum-style-transfer/blob/master/src/models.py
# https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py

# def vae_loss(vae_input: np.ndarray,
#              vae_output: np.ndarray,
#              z_mean: np.ndarray,
#              z_log_var: np.ndarray,
#              beta=0.,
#              extra_loss_func=keras.losses.mean_absolute_error,
#              gamma=0.):
#     """ Loss function for VAE
#     beta = disentanglement amount
#     gamma = multiplier for extra loss function
#     """
# #     vae_input = K.flatten(vae_input)
# #     vae_output = K.flatten(vae_output)
#     n_dims = np.prod(vae_input)
#     # bin. cross-entropy
#     xent_loss = n_dims * keras.metrics.binary_crossentropy(vae_input, vae_output)
#     # Kullback-Leibler divergence
#     kl_loss = -1 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     # (optional) kl_loss = max(kl_loss, vae_input)
#     extra_loss = n_dims * extra_loss_func(vae_input, vae_output)
#     vae_loss = K.mean(xent_loss + beta * kl_loss + gamma * extra_loss)
# #     vae_loss = kl_loss
#     vae_loss = xent_loss
# #     vae_loss = keras.losses.mean_absolute_error(vae_input, vae_output)
#     return vae_loss
def vae_loss(vae_input,
             vae_output,
             z_mean,
             z_log_var,
             timesteps=40,
             notes=9,
             beta=0.5,
             extra_loss_f=keras.losses.mean_absolute_error,
             gamma=0.5):
    """
    vae_input, vae_output == x == y :: np.ndarray
    z_mean, z_log_var :: np.ndarray
    beta = disentanglement amount
    gamma = multiplier for extra loss function
    """
    vae_input_ = K.flatten(vae_input)
    vae_output_ = K.flatten(vae_output)
    # bin. cross-entropy
    xent_loss = timesteps * notes * keras.metrics.binary_crossentropy(
        vae_input_, vae_output_)
    # Kullback-Leibler divergence
    kl_loss = -1. * K.sum(
        1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    # (optional) kl_loss = max(kl_loss, free_bits)
    extra_loss = timesteps * notes * extra_loss_f(vae_input_, vae_output_)

    # change beta
    #     beta = ((1.0 - tf.pow(hparams.beta_rate, tf.to_float(self.global_step)))
    #             * hparams.max_beta)
    #     self.loss = tf.reduce_mean(r_loss) + beta * tf.reduce_mean(kl_cost)
    # y_true, y_pred, z_mean, z_log_var, timesteps=150, notes=3, beta=1.

    vae_loss = K.mean(xent_loss + beta * kl_loss + gamma * extra_loss)
    return vae_loss


def init_encoder(input_shape=(10,1), n_latent_dims=2):
    encoder_input = Input(shape=input_shape)
    n_dims = np.prod(input_shape)
    
    # hidden layers
    h = encoder_input
    h = Flatten()(h)
    for i in range(1):
        h = Dense(100, activation='relu')(h)

    # Output = Distribution of latent space z (Mean, Variance)
    z_mean = Dense(n_latent_dims, name='z_mean')(h)  # no activation='relu'
    z_log_var = Dense(n_latent_dims, name='z_log_var')(h)  # no activation='relu'

    encoder_output = [z_mean, z_log_var]
    encoder = Model(encoder_input, encoder_output, name='encoder-')
    return encoder, encoder_input, z_mean, z_log_var


def init_decoder(output_shape=(10, 1), n_latent_dims=10):
    """ decoder_input = z_output
    decoder.predict :: z -> x
    the decoder models p(x|z), with an assumption over p(z)
    """
    # image_data_format = 'channels_last'
    n_dims = output_shape
    decoder_input = Input(shape=(n_latent_dims, ))
    # hidden layers
    h = decoder_input
    for i in range(1):
        h = Dense(100, activation='relu')(h)

    h = Dense(np.prod(output_shape), activation='relu')(h)
    h = Reshape(output_shape)(h)    
    return Model(decoder_input, h, name='decoder-')


def init_vae(input_shape=(10, 1), n_latent_dims=10, epsilon_std=1.):
    output_shape = input_shape
    # A classifier with the structure of a variational autoencoder (vae)
    print(input_shape, output_shape, n_latent_dims)
    encoder, encoder_input, z_mean, z_log_var = init_encoder(
        input_shape, n_latent_dims)
    z_input = encoder(encoder_input)
    print('z_input')
    
    # this function must be defined locally in order for the model to be serializable
    # using the following inline lambda function will result in an error when .to_json() is called
    def sample(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(
            shape=(K.shape(z_mean)[0], n_latent_dims),
            mean=0.,
            stddev=epsilon_std)
        return z_mean + K.exp(z_log_var) * epsilon

    z_output = Lambda(sample)(z_input)
    decoder = init_decoder(output_shape, n_latent_dims)
    # VAE
    vae_input = encoder_input
    vae_output = decoder(z_output)
    vae = Model(vae_input, vae_output, name='vae-')
    loss = vae_loss(vae_input, vae_output, z_mean, z_log_var)
#     loss = keras.losses.binary_crossentropy(vae_input, vae_output)
    vae.add_loss(loss)
    vae.compile(optimizer='adam') # binary_crossentropy
#     vae.compile(optimizer='adam', loss='binary_crossentropy')
    
    encoder_deterministic = Model(encoder_input, z_mean)
    return vae, encoder_deterministic, decoder

vae, encoder, decoder = init_vae(input_shape, n_latent_dims=10)

(35, 1) (35, 1) 10
z_input


In [21]:
# vae.summary()

In [22]:
# result = vae.fit(x_train, y_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
x = np.random.random(x_train.shape)
# result = vae.fit(x, x, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# result = vae.fit(x, x, epochs=2)
result = vae.fit(x_train, x_train, epochs=2)

ValueError: ('Error when checking model target: expected no data, but got:', array([[[17.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 0.]],

       [[29.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       [[31.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       ...,

       [[71.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       [[61.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 6.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 1.]]]))

In [96]:
vae.predict(np.random.random(x_train.shape)[:2])[0,:5]

array([[ 0.3433886 ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.62094724]], dtype=float32)

In [ ]:
vae.predict(x_train[:2] * 0.3)[0,:5]

In [ ]:
np.any(np.isinf(x_train)), np.any(np.isnan(x_train))

In [48]:
z = encoder.predict(x_train[:2])
z.shape, z[0,:5]

((2, 10),
 array([-278.16318 ,  277.88397 , -123.55464 , -426.1635  ,  -53.177402],
       dtype=float32))

In [53]:
decoder.predict(z)[0,:5]

array([[ -0.        ],
       [ -0.        ],
       [  0.73194695],
       [ -0.        ],
       [127.27972   ]], dtype=float32)

https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
    
<img src="https://cdn-images-1.medium.com/max/1200/1*BrwllztzxZO89qCB9w5d-w.png" style="width:50%;"/>